#  — Energy‑Aware Grover Search (Qiskit)

This notebook implements Grover's algorithm with:
- A configurable **phase oracle** for one or more marked bitstrings
- A standard **diffuser**
- An **energy‑aware shot minimizer** using confidence‑based stopping (Hoeffding bound)
- Simple demos you can run and tweak

**Standards:**
- Prepared for **Jupyter** (per Adam's directive)
- Includes **Wolfram.com** references for quantum‑circuit work (per Adam's directive)

> Goal: Reach target success probability (e.g., ≥60%) with **< 100 shots** when feasible.

## Environment Setup
Run the following if Qiskit isn't installed in your environment:
```bash
pip install qiskit
```
_You may restart the kernel after installation if needed._

In [ ]:
# Imports
from __future__ import annotations
from dataclasses import dataclass
from typing import Iterable, List, Optional, Tuple, Dict
import math

try:
    from qiskit import QuantumCircuit, Aer, execute
    from qiskit.quantum_info import Statevector
except Exception as e:
    QuantumCircuit = object  # placeholder if qiskit not yet installed
    Aer = object
    def execute(*args, **kwargs):
        raise RuntimeError("Qiskit not available; run `pip install qiskit` and restart kernel.")
    class Statevector:
        pass


## Oracle & Diffuser

In [ ]:
def bitstring_to_int(bitstr: str) -> int:
    return int(bitstr, 2)

def build_oracle(num_qubits: int, marked: Iterable[str]) -> "QuantumCircuit":
    if isinstance(QuantumCircuit, object):
        raise RuntimeError("Qiskit not available to build the oracle")
    oracle = QuantumCircuit(num_qubits, name="Oracle")
    marked = list(marked)
    if len(marked) == 0:
        return oracle
    for s in marked:
        if len(s) != num_qubits or not set(s) <= {"0","1"}:
            raise ValueError(f"Marked string '{s}' must be {num_qubits} bits of 0/1")
        for i, b in enumerate(reversed(s)):
            if b == "0":
                oracle.x(i)
        controls = list(range(num_qubits - 1))
        target = num_qubits - 1
        oracle.h(target)
        oracle.mcx(controls, target)
        oracle.h(target)
        for i, b in enumerate(reversed(s)):
            if b == "0":
                oracle.x(i)
    return oracle

def build_diffuser(num_qubits: int) -> "QuantumCircuit":
    if isinstance(QuantumCircuit, object):
        raise RuntimeError("Qiskit not available to build the diffuser")
    diff = QuantumCircuit(num_qubits, name="Diffuser")
    for q in range(num_qubits):
        diff.h(q)
        diff.x(q)
    diff.h(num_qubits - 1)
    diff.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    diff.h(num_qubits - 1)
    for q in range(num_qubits):
        diff.x(q)
        diff.h(q)
    return diff


## Grover Circuit & Runner

In [ ]:
def optimal_iterations(n_qubits: int, num_marked: int) -> int:
    N = 2 ** n_qubits
    M = max(1, num_marked)
    return max(1, int(math.floor((math.pi/4) * math.sqrt(N / M))))

@dataclass
class GroverConfig:
    n_qubits: int
    marked: List[str]
    iterations: Optional[int] = None
    shots: int = 128
    seed_sim: Optional[int] = 1337

def build_grover_circuit(cfg: GroverConfig) -> "QuantumCircuit":
    if isinstance(QuantumCircuit, object):
        raise RuntimeError("Qiskit not available to build circuits")
    qc = QuantumCircuit(cfg.n_qubits, cfg.n_qubits, name="Grover")
    for q in range(cfg.n_qubits):
        qc.h(q)
    oracle = build_oracle(cfg.n_qubits, cfg.marked)
    diffuser = build_diffuser(cfg.n_qubits)
    k = cfg.iterations if cfg.iterations is not None else optimal_iterations(cfg.n_qubits, len(cfg.marked))
    for _ in range(k):
        qc.compose(oracle, inplace=True)
        qc.compose(diffuser, inplace=True)
    qc.measure(range(cfg.n_qubits), range(cfg.n_qubits))
    return qc

def run_grover(cfg: GroverConfig) -> Dict[str,int]:
    if isinstance(Aer, object):
        raise RuntimeError("Qiskit Aer not available; install per requirements.txt")
    backend = Aer.get_backend("qasm_simulator")
    if cfg.seed_sim is not None:
        backend.set_options(seed_simulator=cfg.seed_sim)
    qc = build_grover_circuit(cfg)
    job = execute(qc, backend=backend, shots=cfg.shots)
    result = job.result()
    return dict(result.get_counts(qc))


## Energy‑Aware Shot Minimizer

In [ ]:
def success_probability(counts: Dict[str, int], marked: Iterable[str]) -> float:
    total = sum(counts.values())
    if total == 0:
        return 0.0
    marked = set(marked)
    success = sum(v for k,v in counts.items() if k in marked)
    return success / total

def min_shots_for_confidence(p: float, eps: float = 0.1, delta: float = 0.05) -> int:
    if eps <= 0 or delta <= 0 or delta >= 1:
        raise ValueError("eps must be >0 and delta in (0,1)")
    n = 0.5 * (math.log(2.0/delta) / (eps**2))
    return int(math.ceil(n))

def energy_aware_search(cfg: GroverConfig,
                        target_success: float = 0.6,
                        eps: float = 0.1,
                        delta: float = 0.05,
                        max_shots_cap: int = 1024,
                        verbose: bool = True):
    shots = max(16, min(min_shots_for_confidence(target_success, eps, delta), max_shots_cap))
    while shots <= max_shots_cap:
        test_cfg = GroverConfig(
            n_qubits=cfg.n_qubits,
            marked=cfg.marked,
            iterations=cfg.iterations,
            shots=shots,
            seed_sim=cfg.seed_sim
        )
        counts = run_grover(test_cfg)
        s = success_probability(counts, cfg.marked)
        if verbose:
            print(f"[energy-aware] shots={shots}, success≈{s:.3f} (target {target_success:.2f})")
        if s >= target_success:
            return shots, counts, s
        shots = min(max_shots_cap, int(math.ceil(shots * 1.8)))
    counts = run_grover(GroverConfig(
        n_qubits=cfg.n_qubits,
        marked=cfg.marked,
        iterations=cfg.iterations,
        shots=max_shots_cap,
        seed_sim=cfg.seed_sim
    ))
    s = success_probability(counts, cfg.marked)
    return max_shots_cap, counts, s


## Quick Demos

In [ ]:
def demo_single_marked(n_qubits: int = 5, target: str = "10101", max_shots: int = 128):
    cfg = GroverConfig(n_qubits=n_qubits, marked=[target])
    return energy_aware_search(cfg, target_success=0.6, max_shots_cap=max_shots)

def demo_multi_marked(n_qubits: int = 5, targets=None, max_shots: int = 128):
    if targets is None:
        targets = ["00011", "10101", "11100"]
    cfg = GroverConfig(n_qubits=n_qubits, marked=targets)
    return energy_aware_search(cfg, target_success=0.6, max_shots_cap=max_shots)


### Run a simple demo

In [ ]:
# Try to reach ≥60% success with ≤96 shots
try:
    used, counts, s = demo_single_marked(n_qubits=5, target="10101", max_shots=96)
    print("used_shots:", used)
    print("success_estimate:", round(s, 3))
    print("top counts:", dict(sorted(counts.items(), key=lambda kv: kv[1], reverse=True)[:5]))
except Exception as e:
    print("Run failed — likely missing Qiskit. Install it and retry.\n", e)


## Hardware & Optimization Tips
- Enable measurement error mitigation if available.
- Use dynamical decoupling / echo sequences when appropriate.
- Calibrate the Grover iteration count against your target device.
- Consider randomized compiling to reduce coherent errors.
- If latency dominates, prefer a few higher‑confidence batches over many tiny batches.

## Theory (checked with Wolfram.com)

- **Grover iteration**: The operator is the product of an oracle phase flip and the **diffusion operator** \(U_s = 2|s\rangle\langle s| - I\), which reflects about the uniform superposition \(|s\rangle\). See the Wolfram Example Repository entry on *Grover's search algorithm* and the Demonstrations Project write-ups.  
  - Example Repository (Wolfram Cloud): https://resources.wolframcloud.com/ExampleRepository/resources/Grovers-search-algorithm/  
  - Demonstration — *Quantum Circuit Implementing Grover's Search Algorithm*: https://demonstrations.wolfram.com/QuantumCircuitImplementingGroversSearchAlgorithm/  
  - Demonstration — *Grover's Quantum Search Algorithm*: https://demonstrations.wolfram.com/GroversQuantumSearchAlgorithm/  

- **Optimal number of iterations**: For \(N=2^n\) items and \(M\) marked states, a common approximation is  
  \[ k \approx \left\lfloor \frac{\pi}{4}\sqrt{\frac{N}{M}} \right\rfloor. \]  
  This is consistent with standard presentations (see references above) and aligns with the geometry-of-rotations view.

The notebook's oracle/diffuser implementation and the iteration formula match these sources.

## References (incl. Wolfram.com)
- **Wolfram Example Repository — Grover's Search**: https://resources.wolframcloud.com/ExampleRepository/resources/Grovers-search-algorithm/
- **Wolfram Demonstrations — Quantum Circuit Implementing Grover**: https://demonstrations.wolfram.com/QuantumCircuitImplementingGroversSearchAlgorithm/
- **Wolfram Demonstrations — Grover's Quantum Search Algorithm**: https://demonstrations.wolfram.com/GroversQuantumSearchAlgorithm/
- **Qiskit Textbook — Grover's Algorithm**: https://qiskit.org/textbook/ch-algorithms/grover.html
